# Data Preparation

In [36]:
# import required modules for data preparation tasks
import requests, zipfile, StringIO
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import re
import json
import os

### 1. Get the Data for 2014 from zip Files

First, we want to open and combine the zipped data files for each month that have been downloaded according to the process outlined in `01_Data Aquisition.ipynb`.

In [37]:
# reads all predefined months for a year and merge into one data frame
rawData = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for m in months:
    z = zipfile.ZipFile('cache/{y}{mo}.zip'.format(y=str(2014), mo = m))
    rawData = rawData.append(pd.read_csv(z.open(z.namelist()[0])))
    print "Downloaded", m
# reset index of complete dataset for delays to prepare merging in next step
rawData.reset_index(inplace=True)

Downloaded 01
Downloaded 02
Downloaded 03
Downloaded 04
Downloaded 05
Downloaded 06
Downloaded 07
Downloaded 08
Downloaded 09
Downloaded 10
Downloaded 11
Downloaded 12


### 2. Combine Data with External Aircraft Data

We also have two tables containing infos about the aircraft and its manufacturer available. Both files will be loaded.

In [38]:
z = zipfile.ZipFile('externalData/AircraftInformation.zip')
df_master  = pd.DataFrame.from_csv(z.open('MASTER.txt'))
df_aircrafts  = pd.DataFrame.from_csv(z.open('ACFTREF.txt'))

We can now join these two tables based on their common ID that is saved in the column `MFR MDL CODE` of the master table and in the index of the aircraft table respectively.

In [39]:
master = df_master[['MFR MDL CODE', 'YEAR MFR']].reset_index()
aircrafts = df_aircrafts['MFR'].reset_index()
master.columns = ['TAIL_NUM', 'CODE', 'YEAR']
aircrafts.columns = ['CODE', 'MFR']
joined = pd.merge(master, aircrafts, how='left', on='CODE')

We now join this aircraft information with our delay data and extend the original dataset with the two new features: The year in which the aircraft was built (to determine the age) and the manufacturer.

In [40]:
delayFinal = rawData[['TAIL_NUM','AIRLINE_ID']]
delayFinal.TAIL_NUM = delayFinal.TAIL_NUM.str.strip('N')
delaymfr = pd.merge(delayFinal, joined, how='left', on=['TAIL_NUM'])
rawData['AIRCRAFT_YEAR'] = delaymfr.YEAR
rawData['AIRCRAFT_MFR'] = delaymfr.MFR

### 3. Combine Data with External Airport Location Data

Now we load an external dataset that contains the geolocations for each commercial airport in the world. We filter this to get only the airports in the US and then assign the respective geocode of the origin airport to our original delay dataset by merging both tables.

In [41]:
airportLocation = pd.DataFrame.from_csv('externalData/airport_codes_with_geo_name_ids_and_nl_names-2008-04-14.csv', header=None)
usAirports = airportLocation[airportLocation[4]=='US'].reset_index()
usAirports = usAirports[[0, 5, 6]]
usAirports.columns = ['ORIGIN', 'LAT', 'LONG']
rawData = pd.merge(rawData, usAirports, how='left', on='ORIGIN')

In [42]:
1.0*sum(rawData.LAT.isnull())/rawData.shape[0]

0.007050916258277116

Just 0.7% of alll flight origins could not be located, so the merge was quite successful.

### 4. Combine Data with External Weather Data

As outlined in `01_Data Aquisition.ipynb`, we scraped historical weather data for major US airports from the web. This data can be used as additional features for each flight to get information about the current weather conditions at the airport of the departure.

In [43]:
weatherFile = os.path.join('cache', 'weather_data.json')
with open(weatherFile) as infile:
    weatherDict = json.load(infile)

In [44]:
dates = []
frames = []
for datapoint in weatherDict['JFK']:
    date = datapoint['date']
    frames.append(pd.DataFrame(datapoint['data'], index=['%s-%s-%s' % (date[0:4], date[4:6], date[6:8])]))
weather_df = pd.concat(frames).reset_index()

In [45]:
# weather_df.head()
jfk_delays = rawData[rawData.ORIGIN=='JFK']

In [46]:
jfk_dalayWeather = pd.merge(jfk_delays, weather_df, how='left', left_on='FL_DATE', right_on = 'index')

In [47]:
jfk_dalayWeather.to_csv('cache/jfk_weather_2014.csv', encoding='UTF-8')

### 5. Creation of the Final Dataset

The columns we now have in the dataset are:

In [48]:
rawData.columns

Index([u'index', u'YEAR', u'QUARTER', u'MONTH', u'DAY_OF_MONTH',
       u'DAY_OF_WEEK', u'FL_DATE', u'UNIQUE_CARRIER', u'AIRLINE_ID',
       u'CARRIER', u'TAIL_NUM', u'FL_NUM', u'ORIGIN', u'ORIGIN_CITY_NAME',
       u'ORIGIN_STATE_ABR', u'ORIGIN_STATE_FIPS', u'ORIGIN_STATE_NM',
       u'ORIGIN_WAC', u'DEST', u'DEST_CITY_NAME', u'DEST_STATE_ABR',
       u'DEST_STATE_FIPS', u'DEST_STATE_NM', u'DEST_WAC', u'CRS_DEP_TIME',
       u'DEP_TIME', u'DEP_DELAY', u'DEP_DELAY_NEW', u'DEP_DEL15',
       u'DEP_DELAY_GROUP', u'DEP_TIME_BLK', u'TAXI_OUT', u'WHEELS_OFF',
       u'WHEELS_ON', u'TAXI_IN', u'CRS_ARR_TIME', u'ARR_TIME', u'ARR_DELAY',
       u'ARR_DELAY_NEW', u'ARR_DEL15', u'ARR_DELAY_GROUP', u'ARR_TIME_BLK',
       u'CANCELLED', u'CANCELLATION_CODE', u'DIVERTED', u'CRS_ELAPSED_TIME',
       u'ACTUAL_ELAPSED_TIME', u'AIR_TIME', u'FLIGHTS', u'DISTANCE',
       u'DISTANCE_GROUP', u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY',
       u'SECURITY_DELAY', u'LATE_AIRCRAFT_DELAY', u'FIRST_DEP_TI

However, we just need a subset of these columns for our analysis:

In [49]:
selectedColumns = [u'index', u'FL_DATE', u'UNIQUE_CARRIER', u'TAIL_NUM', u'FL_NUM', 
                   u'ORIGIN', u'DEST', u'CRS_DEP_TIME', u'DEP_TIME', u'DEP_DELAY', u'TAXI_OUT', 
                   u'WHEELS_OFF', u'WHEELS_ON', u'TAXI_IN', u'CRS_ARR_TIME', u'ARR_TIME', u'ARR_DELAY', 
                   u'CANCELLED', u'DIVERTED', u'CANCELLATION_CODE', u'AIR_TIME', u'DISTANCE', 
                   u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY', u'SECURITY_DELAY', u'LATE_AIRCRAFT_DELAY', 
                   u'AIRCRAFT_YEAR', u'AIRCRAFT_MFR',u'LAT', u'LONG']

In [50]:
complete2014Data = rawData[selectedColumns]

The resulting dataframe `complete2014Data` will be locally stored as csv file.

In [51]:
complete2014Data.to_csv('cache/complete2014Data.csv')

### 6. Creation of the Prediction Datasets

Before evaluating any models on the data, we have to clean it a bit.

### Pruning the data

When cleaning the data set, we have to remove the following entries:

- flights that have been cancelled or diverted. We focus on predicting the delay. As a result, we also remove the columns associated with diverted flights.
- colmuns that give the answer. This is the case of many colmuns related to the arrival of the plane
- rows where a value is missing

Note that data points have to be cleaned in this order because most flights have empty entries for the 'diverted' columns.

In [69]:
#entries to be dropped in the analysis
flight_data_dropped = ['QUARTER', 'DAY_OF_MONTH', 'AIRLINE_ID', 'CARRIER', 'FL_NUM', 'TAIL_NUM']

location_data_dropped = ['ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM',\
                    'ORIGIN_WAC', 'DEST_STATE_FIPS', \
                    'DEST_STATE_NM', 'DEST_WAC']

departure_data_dropped = ['DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP',\
                          'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF']

arrival_data_dropped = ['WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY_NEW',\
                       'ARR_DELAY_GROUP', 'ARR_TIME_BLK']

cancel_data_dropped = ['CANCELLED','CANCELLATION_CODE', 'DIVERTED']

summaries_dropped = ['CRS_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS']

cause_delay_dropped = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']

gate_return_dropped = ['FIRST_DEP_TIME', 'TOTAL_ADD_GTIME', 'LONGEST_ADD_GTIME']

diverted_data_dropped = ['DIV_AIRPORT_LANDINGS', 'DIV_REACHED_DEST', 'DIV_ACTUAL_ELAPSED_TIME',  \
                         'DIV_ARR_DELAY', 'DIV_DISTANCE', 'DIV1_AIRPORT', 'DIV1_WHEELS_ON', \
                         'DIV1_TOTAL_GTIME', 'DIV1_LONGEST_GTIME', 'DIV1_WHEELS_OFF', \
                         'DIV1_TAIL_NUM', 'DIV2_AIRPORT', 'DIV2_WHEELS_ON', \
                         'DIV2_TOTAL_GTIME', 'DIV2_LONGEST_GTIME', 'DIV2_WHEELS_OFF', \
                         'DIV2_TAIL_NUM', 'DIV3_AIRPORT', 'DIV3_WHEELS_ON', \
                         'DIV3_TOTAL_GTIME', 'DIV3_LONGEST_GTIME', 'DIV3_WHEELS_OFF', 'DIV3_TAIL_NUM', \
                         'DIV4_AIRPORT', 'DIV4_WHEELS_ON', 'DIV4_TOTAL_GTIME', 'DIV4_LONGEST_GTIME', \
                         'DIV4_WHEELS_OFF', 'DIV4_TAIL_NUM', 'DIV5_AIRPORT', 'DIV5_WHEELS_ON', \
                         'DIV5_TOTAL_GTIME', 'DIV5_LONGEST_GTIME', 'DIV5_WHEELS_OFF', 'DIV5_TAIL_NUM']

other_dropped = ['Unnamed: 93']

columns_dropped = flight_data_dropped + location_data_dropped + departure_data_dropped + arrival_data_dropped \
    + cancel_data_dropped + summaries_dropped + cause_delay_dropped + gate_return_dropped + diverted_data_dropped \
    + other_dropped

In [70]:
def clean(data, list_col):
    ''' 
    Creates a dataset by excluding undesirable columns

    Parameters:
    -----------

    df: pandas.DataFrame
       Flight dataframe  

    list_col: <list 'string'>
        Comumns to exclude from the data set
    '''

    # security check to drop only columns that exist
    list_col = list(set(list_col) & set(data.columns))
    
    res = data[(data.CANCELLED == 0) & (data.DIVERTED == 0)]
    res.drop(list_col, axis=1, inplace=True)
    res.dropna(axis = 0, inplace = True)
    return res

In [71]:
%%time
data2014 = clean(complete2014Data, columns_dropped)
print data2014.columns

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index([u'index', u'FL_DATE', u'UNIQUE_CARRIER', u'ORIGIN', u'DEST',
       u'CRS_DEP_TIME', u'CRS_ARR_TIME', u'ARR_DELAY', u'DISTANCE',
       u'AIRCRAFT_YEAR', u'AIRCRAFT_MFR', u'LAT', u'LONG'],
      dtype='object')
CPU times: user 3.99 s, sys: 1.7 s, total: 5.68 s
Wall time: 6.13 s


[u'MONTH', u'DAY_OF_WEEK', u'UNIQUE_CARRIER', u'ORIGIN',
       u'ORIGIN_CITY_NAME', u'ORIGIN_STATE_ABR', u'DEST', u'DEST_CITY_NAME',
       u'DEST_STATE_ABR', u'CRS_DEP_TIME', u'CRS_ARR_TIME', u'ARR_DEL15',
       u'DISTANCE', u'DISTANCE_GROUP']

In [72]:
data2014.head()

,index,FL_DATE,UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,DISTANCE,AIRCRAFT_YEAR,AIRCRAFT_MFR,LAT,LONG
0,0,2014-01-01,AA,JFK,LAX,900,1225,13,2475,1987,BOEING,40.633333,-73.783333
1,1,2014-01-02,AA,JFK,LAX,900,1225,1,2475,1987,BOEING,40.633333,-73.783333
3,3,2014-01-04,AA,JFK,LAX,900,1225,59,2475,1986,BOEING,40.633333,-73.783333
5,5,2014-01-06,AA,JFK,LAX,900,1225,-8,2475,1985,BOEING,40.633333,-73.783333
8,8,2014-01-09,AA,JFK,LAX,900,1225,-21,2475,1988,BOEING,40.633333,-73.783333


In [73]:
# save data to cache
data2014.to_csv('cache/linear_model_data.csv')